# Reproducing Vitaly Feldman MNIST Results

The goal of this notebook is to reproduce the results in https://arxiv.org/abs/2008.11193 which are based on the MNIST model from https://github.com/pytorch/opacus/blob/master/examples/mnist.py

In [11]:
import numpy as np
import syft as sy
from syft.core.tensor.tensor import Tensor
from syft.core.adp.entity import Entity
from syft.core.adp.adversarial_accountant import AdversarialAccountant
from collections import defaultdict

data_batch = np.random.rand(4,2)
label_batch = np.random.rand(4,10)  

class DataSubjectGuardianOfTheGalaxy():
    """One of these per domain"""
    
    def __init__(self, enforce_unique_attrs=['name', 'email']):
        """"""
        self.enforce_unique_attrs = enforce_unique_attrs
        self.entity_id2obj = {}
        
        self.unique_attr2entity_id = defaultdict(dict)
        
        # verfiy_key-to-accountant mapping
        self.accountants = {}
        
        # the idea with the symbol factory is that a session with a data scientist
        # will have a unique SymbolFactory for the length of that session (this is
        # different from an accountant which a data scientist is stuck with for life).
        # A Symbol Factory is an object which produces symbols (mapped to prime numbers)
        # whenever a PhiTensor is turned into a GammaTensor. Thus, the Symbol Factory
        # can be re-initialized every time a data scientist creates a new session on a worker. However,
        # if ever the data scientist were to save some intermediate variables across sessions or,
        # to transfer a variable from one of their workers to another,
        # they would also need to bring across the symbol factory associated with those variables.
        # The main issue is that a symbol factory is what allows the privacy logic in gammatensor to
        # know when it's dealing with the same symbol twice or two different symbols. 
        
    def register_data_subject(self, **attributes):
        """"""
        
        for attr in self.enforce_unique_attrs:
            if attr not in attributes.keys():
                raise Exception("The DataSubjectGuardian for your node requires you to "+\
                                "specify a unique " + str(attr) + " for every data subject. "+\
                               "Please specify email=<their email> in register_entity() method.")
        
        ent = Entity(**attributes)

        for attr in self.enforce_unique_attrs:
            try:
                self.unique_attr2entity_id[attr][ent.attributes[attr]] = ent.id
            except KeyError as e:
                print(e)
                raise KeyError("'" + str(attr) + "' attribute must be unique '" + str(attributes[attr]) + \
                               "' has already been claimed by another entity.")
                
        return ent
    
    @property
    def symbol_factory(self):
        return self._symbol_factory

import sympy as sp    
class PrimeFactory:

    """IMPORTANT: it's very important that two tensors be able to tell that
    they are indeed referencing the EXACT same PrimeFactory. At present this is done
    by ensuring that it is literally the same python object. In the future, we will probaby
    need to formalize this. However, the main way this could go wrong is if we created some
    alternate way for checking to see if two prime factories 'sortof looked the same' but which
    in fact weren't the EXACT same object. This could lead to security leaks wherein two tensors
    think two different symbols in fact are the same symbol."""

    def __init__(self):
        self.prev_prime = 1

    def next(self):
        self.prev_prime = sp.nextprime(self.prev_prime)
        return self.prev_prime
    
from syft.core.adp.scalar import GammaScalar
    
class VirtualMachinePrivateScalarManager:

    def __init__(self):
        self.prime_factory = PrimeFactory()
        self.prime2symbol = {}
        
    def get_symbol(self,min_val, value, max_val, entity):
        gs = GammaScalar(min_val=min_val,
                        value=value,
                        max_val=max_val,
                        entity=entity)
        gs.prime = self.prime_factory.next()
        self.prime2symbol[gs.prime] = gs
        return gs.prime

    
dsg = DataSubjectGuardianOfTheGalaxy()
bob = dsg.register_data_subject(name="Bob", email="bob@openmined.org")
kritika = dsg.register_data_subject(name="Kritika", email="kritika@openmined.org")
madhava = dsg.register_data_subject(name="Madhava", email="madhava@openmined.org")
tudor = dsg.register_data_subject(name="Tudor", email="tudor@openmined.org")

entities = [bob,kritika,madhava,tudor]

# needs to come from the VirtualWorker/Session/etc.
scalar_manager = VirtualMachinePrivateScalarManager()

data = Tensor(data_batch).private(0.01,1,entities=entities, scalar_manager=scalar_manager).autograd(requires_grad=True)
data2 = Tensor(data_batch).private(0.02,2,entities=[bob,bob,bob,bob], scalar_manager=scalar_manager).autograd(requires_grad=True)

In [12]:
y = data.sum(0)
y2 = data2.sum(0)

In [13]:
acc = AdversarialAccountant(20)

In [14]:
y.child.child.flat_scalars

[<IntermediateGammaScalar: (0.04 < 3.278920885978709 < 4.0)>,
 <IntermediateGammaScalar: (0.04 < 2.3434082360030635 < 4.0)>]

In [18]:
_y = y.publish(acc=acc, sigma=1.5)
# _y2 = y2.publish(acc=acc, sigma=1.5)
acc.print_ledger()

<Entity:Tudor>	16.754948874946987
<Entity:Kritika>	16.754948874946987
<Entity:Madhava>	16.754948874946987
<Entity:Bob>	16.754948874946987


In [3]:
from syft.core.tensor.passthrough import HANDLED_FUNCTIONS

In [4]:

data_batch = np.random.rand(4,28*28)
label_batch = np.random.rand(4,10)  

bob = Entity(unique_name="Bob")

data = Tensor(data_batch).autograd(requires_grad=True)
target = Tensor(label_batch).autograd(requires_grad=True)

weights = Tensor(np.random.rand(28*28,10)).autograd(requires_grad=True)

for i in range(10):
    pred = data.dot(weights)
    diff = target - pred
    pre_loss = np.square(diff)
    loss = np.mean(pre_loss)
    extraneous_thing = -diff
    loss.backward()

    weights = weights - (weights.grad * 0.01)
    print(loss)

Tensor(child=AutogradTensor(child=[39593.85306277]))
Tensor(child=AutogradTensor(child=[12606.85187116]))
Tensor(child=AutogradTensor(child=[4015.70540841]))
Tensor(child=AutogradTensor(child=[1280.66337406]))
Tensor(child=AutogradTensor(child=[409.85003185]))
Tensor(child=AutogradTensor(child=[132.50026923]))
Tensor(child=AutogradTensor(child=[44.08106506]))
Tensor(child=AutogradTensor(child=[15.81375355]))
Tensor(child=AutogradTensor(child=[6.70270769]))
Tensor(child=AutogradTensor(child=[3.69698355]))


In [1]:
import pymbolic as pmbl
import numpy as np

vs = list()
for i in range(2*28*28):
    x = pmbl.var("s"+str(i))
    vs.append(x)
    
ws = list()
for i in range(10*28*28):
    w = pmbl.var("w"+str(i))
    ws.append(x)    
    
data = np.array(vs).reshape(2,28*28)
weights = np.array(ws).reshape(28*28,10)

In [2]:
%%timeit
pred = data.dot(weights)

58.3 ms ± 270 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
from syft.lib.adp.tensor import Tensor
from syft.lib.adp.tensor import make_entities


data = Tensor(np.random.rand(2,28*28))
target = Tensor(np.random.rand(28*28,10))
entities = make_entities(n=len(data))
data = data.private(min_val=-0.42421296, max_val=2.8214867, entities=entities)
data = data.reshape(2,28*28)
weights = np.random.rand(28 * 28, 10)

making entities


In [4]:
%%timeit -n1 -r1
out = data.dot(weights)

467 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [1]:
import cocos.device as cd
from cocos.symbolic import (
    LambdifiedMatrixExpression, \
    find_length_of_state_vectors
)
cd.info()
from cocos import numerics as cn

Cocos running on ArrayFire v3.5.1 (OpenCL 64bit)
[0] Apple: AMD Radeon Pro Vega 56 Compute Engine | OpenCL | compute version 1.2


In [14]:
data = cn.random.rand(500000,28*28)
weights = cn.random.rand(28*28,10)

In [15]:
%%timeit -n1 -r1
out = data.dot(weights)

297 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


##### out

In [54]:
out2 = np.dot(data,weights)

In [62]:
sp = out2[0][0].value.poly

In [ ]:
class ScalarTensor(np.ndarray):
    def __new__(
        cls,
        data
    ):
        obj = np.asarray(data).view(cls)
        return obj
    
    def __array_finalize__(self, obj):
        print("finalizing")
        if obj is None:
            return

    def __array_wrap__(self, out_arr, context=None):
        
        print(context)
        
        # out_arr.view calls __array_finalize__
        output = out_arr.view(self.__class__)
        
        return output

In [4]:
import cocos.device as cd
from cocos.symbolic import (
    LambdifiedMatrixExpression, \
    find_length_of_state_vectors
)
cd.info()

Cocos running on ArrayFire v3.5.1 (OpenCL 64bit)
[0] Apple: AMD Radeon Pro Vega 56 Compute Engine | OpenCL | compute version 1.2


In [5]:
import sympy as sym
import numpy as np

In [8]:
# x1, x2, x3, t = sym.symbols('x1, x2, x3, t')
# argument_symbols = (x1, x2, x3)
# g = sym.Function('g')
# f = sym.Matrix([[x1 + x2], [(g(t) * x1 + x3) ** 2], [sym.exp(x1 + x2 + g(t))]])
# jacobian_f = f.jacobian([x1, x2, x3])

# def numeric_time_function(t: float):
#     return np.log(t)

# jacobian_f_lambdified \
#     = LambdifiedMatrixExpression(
#         argument_symbols=argument_symbols,
#         time_symbol=t,
#         symbolic_matrix_expression=jacobian_f,
#         symbolic_time_function_name_to_numeric_time_function_map={'g': numeric_time_function})

# n = 10000000
# X_gpu = cn.random.rand(n, 3)
# X_cpu = np.array(X_gpu)

In [9]:
# %%timeit
# result = X_gpu.sum()

In [10]:
# %%timeit
# result2 = X_cpu.sum()

In [37]:
single_poly = np.array([1,0,1])

single_poly.shape

(3,)

In [14]:
n_polys = (np.random.rand(10,3) > 0.5).astype(int)
n_polys

array([[1, 0, 0],
       [0, 1, 1],
       [0, 1, 1],
       [0, 1, 1],
       [0, 1, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 1],
       [0, 0, 0],
       [1, 1, 0]])

In [12]:
x = data.flatten()[0:5]

In [1]:
import autograd.numpy as np   # Thinly-wrapped version of Numpy
from autograd import grad

def taylor_sine(x):  # Taylor approximation to sine function
    ans = currterm = x
    i = 0
    while np.abs(currterm) > 0.001:
        currterm = -currterm * x**2 / ((2 * i + 3) * (2 * i + 2))
        ans = ans + currterm
        i += 1
    return ans

grad_sine = grad(taylor_sine)
print("Gradient of sin(pi) is", grad_sine(x))

NameError: name 'x' is not defined

In [8]:
# %%timeit
# jacobian_f_numeric_gpu = \
#     (jacobian_f_lambdified
#      .evaluate_with_kwargs(x1=X_gpu[:, 0],
#                            x2=X_gpu[:, 1],
#                            x3=X_gpu[:, 2],
#                            t=1.0))

In [9]:
# %%timeit
# jacobian_f_numeric_cpu = \
#     (jacobian_f_lambdified
#      .evaluate_with_kwargs(x1=X_cpu[:, 0],
#                            x2=X_cpu[:, 1],
#                            x3=X_cpu[:, 2],
#                            t=1.0))

In [21]:
print(f'numerical results from cpu and gpu match: '
      f'{np.allclose(jacobian_f_numeric_gpu, jacobian_f_numeric_cpu)}')

numerical results from cpu and gpu match: True


In [7]:
import symengine
from symengine import var

In [8]:
import uuid

In [9]:
syms = ""
for i in range(784):
    id = str(uuid.uuid4()).replace("-","")
    syms += 's'+id+str(i)+' '

In [10]:
v = var(syms)

In [11]:
%%timeit -n1 -r1
result = np.sum(v)

63.8 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


58 ms ± 291 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [28]:
%%timeit -n1 -r1
out = np.sum(vs)

2.12 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [68]:
syms = ""
for i in range(784):
    id = str(uuid.uuid4()).replace("-","")
    syms += 's'+id+str(i)+','

In [69]:
from sympy import symbols

In [70]:
symbols = symbols(syms[:-1])

In [71]:
%%timeit -n1 -r1
out = np.sum(symbols)

2.01 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [2]:
%%timeit -n1 -r1
out = np.sum([[x.value.poly for x in row] for row in data[0].tolist()])

2.05 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [3]:
# %%timeit -n1 -r1
# result = data[0].sum()

In [4]:
%%timeit -n1 -r1
result = data[1].sum()

2.06 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [5]:
%%timeit -n1 -r1
result = data[0].sum()

2.07 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [6]:
%%timeit -n1 -r1
result = data[1].sum()

2.06 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [5]:
acc = AdversarialAccountant(max_budget=7000)

In [6]:
out = result.publish(acc=acc,sigma=1.5)
acc.print_ledger()

KeyboardInterrupt: 

In [48]:
acc.print_ledger()

<Entity:Constance_Weddle>	2.365443354618569


In [6]:
=

x = Tensor(np.array([[1,1],[1,0],[0,1],[0,0]])).private(min_val=0, max_val=1, entities=entities, is_discrete=True)
y = Tensor(np.array([[1],[1],[0],[0]])).private(min_val=0, max_val=1, entities=entities, is_discrete=False)

_weights = Tensor(np.random.uniform(size=(2,1)))

SyntaxError: invalid syntax (<ipython-input-6-07842543fea2>, line 1)

In [2]:
weights = _weights + 0
# acc = AdversarialAccountant(max_budget=7)  # causes error at end of budget


for i in range(3):
    batch_loss = 0

    pred = x.dot(weights)
    pre_loss = np.square(y-pred)
    loss = np.mean(pre_loss, axis=1)
#     loss = np.mean(pre_loss)    
    ledger = loss.backward(accumulate_grads=True)
    weight_grad = (weights.grad*0.3)
    weight_grad_noise = weight_grad.publish(acc=acc, sigma=0.005)
    weights = weights - weight_grad_noise
    ledger.zero_grads()
    batch_loss += loss.value
    print(np.sum([loss.value]))
    
acc.print_ledger()


0.17169736596498625
0.07441205306476974
0.04001946718047936
<Entity:George>	602.4247883171074
<Entity:Kritika>	602.4247883171074
<Entity:Madhava>	602.4247883171074
<Entity:Tudor>	602.4247883171074


In [13]:
x = PhiScalar(0,0.01,1)

In [14]:
out = x * x

In [15]:
out.publish(acc=acc)

<IntermediatePhiScalar: (0.0 < 0.000100000000000000 < 1.0)>
found one!


[0.703353940208477]

In [10]:
acc.print_ledger()

<Entity:George>	0.0
<Entity:Richard_Alvarez>	0.0002737137892581691


In [1]:
x.entity

NameError: name 'x' is not defined

In [7]:
import sympy as sym

In [9]:
a,b = sym.symbols('a,b')

In [10]:
y = a + b

In [38]:
from sympy.core.numbers import Number
class Float2(Number):
    
    def __init__(self, val):
        self.val = val
        
    def __add__(self, other):
        print("adding")
        return Float(self.val - other.val)

In [39]:
out = y.subs({a:Float2(3)})
out

b + 3

In [40]:
out.subs({b:Float2(2)})

5

In [9]:
# loss[0].value.poly

IndexError: too many indices for array

24ffce1fe8ab44a583abecf0b2e1ca69_c06e7e291fdb4cb6b5ec0b9abbabf1bf + 3*3175e5ebc4944304bb4a1a010fa5003d_162afbde9a87417a9a49c14568ea1c11 + 4*5ec1e3788c314340a6ea45342375281e_162afbde9a87417a9a49c14568ea1c11 + 6*68ed3172c1f6417faa4f5ca24611e12d_d59083b02ef74bdf8122e0c51454aa71 + 6*98d310c205b743be9af907ff4646238b_b63effdec6ba47bf93a126d0b5cc7086 + d082ac7660e84d0c806997c2dbe4f600_b63effdec6ba47bf93a126d0b5cc7086 + 5*eeffa282abe44326ae5fc1b8a6f8c6a0_d59083b02ef74bdf8122e0c51454aa71 + 2*fdefb3e070394d5cbf22820ac0ab7b00_c06e7e291fdb4cb6b5ec0b9abbabf1bf

In [ ]:
## first-pass methods to try to get into AutogradTensor

ops = ['T',
 '__abs__',
#  '__add__',
#  '__and__',
#  '__array__',
#  '__array_finalize__',
#  '__array_function__',
#  '__array_interface__',
#  '__array_prepare__',
#  '__array_priority__',
#  '__array_struct__',
#  '__array_ufunc__',
#  '__array_wrap__',
#  '__bool__',
#  '__class__',
#  '__complex__',
#  '__contains__',
#  '__copy__',
#  '__deepcopy__',
#  '__delattr__',
#  '__delitem__',
#  '__dir__',
 '__divmod__',
#  '__doc__',
 '__eq__',
#  '__float__',
 '__floordiv__',
#  '__format__',
 '__ge__',
#  '__getattribute__',
 '__getitem__',
 '__gt__',
#  '__hash__',
#  '__iadd__',
#  '__iand__', # trigger an error for inline 
#  '__ifloordiv__',
#  '__ilshift__',
#  '__imatmul__',
#  '__imod__',
#  '__imul__',
 '__index__',
#  '__init__',
#  '__init_subclass__',
#  '__int__',
 '__invert__',
#  '__ior__',
#  '__ipow__',
#  '__irshift__',
#  '__isub__',
 '__iter__',
#  '__itruediv__',
#  '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
#  '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
#  '__new__',
#  '__or__',
#  '__pos__',
 '__pow__',
 '__radd__',
#  '__rand__',
#  '__rdivmod__',
#  '__reduce__',
#  '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rlshift__',
 '__rmatmul__',
#  '__rmod__',
 '__rmul__',
#  '__ror__',
 '__rpow__',
 '__rrshift__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
#  '__rxor__',
#  '__setattr__',
#  '__setitem__',
#  '__setstate__',
 '__sizeof__',
 '__str__',
 '__sub__',
#  '__subclasshook__',
 '__truediv__',
#  '__xor__',
#  'all',
#  'any',
 'argmax',
 'argmin',
#  'argpartition',
 'argsort',
#  'astype',
#  'base',
#  'byteswap',
 'choose',
 'clip',
#  'compress',
#  'conj',
#  'conjugate',
 'copy',
#  'ctypes',
 'cumprod',
 'cumsum',
#  'data',
 'diagonal',
 'dot',
#  'dtype',
#  'dump',
#  'dumps',
#  'fill',
#  'flags',
 'flat',
 'flatten',
#  'getfield',
#  'imag',
 'item',
 'itemset',
 'itemsize',
 'max',
 'mean',
 'min',
#  'nbytes',
 'ndim',
#  'newbyteorder',
#  'nonzero',
#  'partition',
 'prod',
#  'ptp',
#  'put',
#  'ravel',
#  'real',
 'repeat',
 'reshape',
 'resize',
#  'round',
#  'searchsorted',
#  'setfield',
#  'setflags',
#  'shape',
#  'size',
 'sort',
 'squeeze',
 'std',
#  'strides',
 'sum',
 'swapaxes',
 'take',
#  'tobytes',
#  'tofile',
#  'tolist',
#  'tostring',
#  'trace',
 'transpose',
#  'var',
#  'view'
      ]